# 🎂 Cake AI Image Generator

This notebook generates realistic cake images based on customer text descriptions.

## Setup Instructions:
1. **Enable GPU**: Go to `Runtime` → `Change runtime type` → Select `T4 GPU`
2. **Run all cells** in order (Ctrl+F9 or Runtime → Run all)
3. **Modify the prompts** in the last cell to generate your custom cakes!

---

## Step 1: Install Required Libraries

In [ ]:
# Install AI image generation libraries
!pip install diffusers transformers accelerate safetensors -q
!pip install xformers -q
print("✅ Libraries installed successfully!")

## Step 2: Load the AI Model

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from IPython.display import display
import os

# Check GPU availability
print(f"🔧 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Load Stable Diffusion model
print("\n🎂 Loading Cake AI Model... (first time takes 2-3 minutes)")

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None
)

# Use faster scheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Move to GPU and enable memory optimizations
pipe = pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

print("\n✅ Model loaded successfully! Ready to generate cakes! 🎉")

## Step 3: Define Cake Generator Function

In [ ]:
def generate_cake(customer_request, save_path=None):
    """
    Generate a cake image based on customer description.
    
    Args:
        customer_request: Description of the cake (e.g., "chocolate cake with roses")
        save_path: Optional path to save the image
    
    Returns:
        Generated PIL Image
    """
    
    # Enhance prompt for better cake images
    prompt = f"""Professional bakery product photo of {customer_request}, 
    highly detailed, realistic cake, delicious looking, 
    studio lighting, 4k quality, food photography, 
    bakery display, appetizing"""
    
    negative_prompt = """cartoon, blurry, low quality, distorted, 
    unrealistic, bad anatomy, watermark, text, logo, 
    ugly, deformed, disfigured, overexposed, underexposed"""
    
    print(f"\n🎨 Generating: {customer_request}")
    print("⏳ Please wait... (about 10-15 seconds)")
    
    # Generate image
    image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        width=512,
        height=512
    ).images[0]
    
    # Save if path provided
    if save_path:
        image.save(save_path)
        print(f"💾 Saved to: {save_path}")
    
    print("✅ Done!")
    return image


def generate_multiple_cakes(customer_request, num_variations=4):
    """
    Generate multiple variations of a cake design.
    Useful for giving customers options to choose from.
    """
    print(f"\n🎂 Generating {num_variations} variations of: {customer_request}")
    
    images = []
    for i in range(num_variations):
        print(f"\n--- Variation {i+1}/{num_variations} ---")
        img = generate_cake(customer_request, f"cake_variation_{i+1}.png")
        images.append(img)
    
    return images

print("✅ Cake generator functions ready!")

---
## 🎂 Step 4: Generate Your Cakes!

**Modify the prompts below to match what your customers want!**

In [ ]:
# ========================================
# 🎂 GENERATE A SINGLE CAKE
# ========================================
# Change this description to whatever your customer wants!

customer_request = "a 3-tier wedding cake with pink roses and gold accents"

cake_image = generate_cake(customer_request, "my_cake.png")
display(cake_image)

In [ ]:
# ========================================
# 🎂 TRY DIFFERENT CAKE STYLES
# ========================================

cake_styles = [
    "chocolate birthday cake with colorful sprinkles and 5 candles",
    "elegant white fondant cake with butterfly decorations",
    "red velvet cake with cream cheese frosting and strawberries",
    "unicorn themed rainbow cake with gold horn",
]

# Create output folder
os.makedirs("generated_cakes", exist_ok=True)

for i, style in enumerate(cake_styles):
    img = generate_cake(style, f"generated_cakes/cake_{i+1}.png")
    display(img)
    print("\n" + "="*50 + "\n")

In [ ]:
# ========================================
# 🎂 INTERACTIVE MODE - TYPE YOUR OWN!
# ========================================

# Change this to any cake description you want
my_custom_cake = "luxury black forest cake with cherries and chocolate shavings"

result = generate_cake(my_custom_cake)
display(result)

In [ ]:
# ========================================
# 🎂 GENERATE MULTIPLE VARIATIONS
# ========================================
# This gives customers 4 different options to choose from!

customer_order = "baby shower cake with blue and white theme"
variations = generate_multiple_cakes(customer_order, num_variations=4)

# Display all variations side by side
from PIL import Image
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
for i, (ax, img) in enumerate(zip(axes.flat, variations)):
    ax.imshow(img)
    ax.set_title(f"Option {i+1}")
    ax.axis('off')
plt.tight_layout()
plt.savefig("cake_options.png", dpi=150)
plt.show()

print("\n🎉 All cake options saved to 'cake_options.png'!")

---
## 📥 Download Generated Images

Run this cell to download all generated cakes to your computer.

In [ ]:
# Download generated images
from google.colab import files
import glob

# Download single cakes
for png in glob.glob("*.png"):
    print(f"📥 Downloading: {png}")
    files.download(png)

# Download from generated_cakes folder
for png in glob.glob("generated_cakes/*.png"):
    print(f"📥 Downloading: {png}")
    files.download(png)

print("\n✅ All images downloaded!")

---
## 💡 Tips for Better Results

**Good prompt examples:**
- "elegant 2-tier wedding cake with white fondant and gold leaf decorations"
- "cute dinosaur birthday cake for a 5 year old with green frosting"
- "rustic naked cake with fresh berries and flowers"
- "minimalist Korean style cake with buttercream flowers"

**Add details like:**
- Number of tiers
- Colors
- Decorations (flowers, fondant figures, etc.)
- Theme (wedding, birthday, baby shower)
- Style (modern, rustic, elegant, cute)

---
Made for Cake Ordering App 🍰